# Python API Example - Netbacks Data Import
## Importing Netbacks Data into a Pandas DataFrame

Here we import Netbacks data from the Spark Python API. 

We then store in a DataFrame for easy exploration and filtering.

This guide is designed to provide an example of how to access the Spark API:
- The path to your client credentials is the only input needed to run this script (just before Section 2)
- This script has been designed to display the raw outputs of requests from the API, and how to format those outputs to enable easy reading and analysis
- This script can be copy and pasted by customers for quick use of the API
- Once comfortable with the process, you can change the variables that are called to produce your own custom analysis products. (Section 2 onwards in this guide).

__N.B. This guide is just for Netbacks data. If you're looking for other API data products (such as Price releases or Freight routes), please refer to their according code example files.__ 

### Have any questions?

If you have any questions regarding our API, or need help accessing specific datasets, please contact us at:

__data@sparkcommodities.com__

or refer to our API website for more information about this endpoint:
https://www.sparkcommodities.com/api/request/netbacks.html

## 1. Importing Data

Here we define the functions that allow us to retrieve the valid credentials to access the Spark API.

__This section can remain unchanged for most Spark API users.__

In [35]:
# Importing libraries for calling the API
import json
import os
import sys
import pandas as pd
from base64 import b64encode
from urllib.parse import urljoin


try:
    from urllib import request, parse
    from urllib.error import HTTPError
except ImportError:
    raise RuntimeError("Python 3 required")

In [36]:
# Defining functions for API request

API_BASE_URL = "https://api.sparkcommodities.com"


def retrieve_credentials(file_path=None):
    """
    Find credentials either by reading the client_credentials file or reading
    environment variables
    """
    if file_path is None:
        client_id = os.getenv("SPARK_CLIENT_ID")
        client_secret = os.getenv("SPARK_CLIENT_SECRET")
        if not client_id or not client_secret:
            raise RuntimeError(
                "SPARK_CLIENT_ID and SPARK_CLIENT_SECRET environment vars required"
            )
    else:
        # Parse the file
        if not os.path.isfile(file_path):
            raise RuntimeError("The file {} doesn't exist".format(file_path))

        with open(file_path) as fp:
            lines = [l.replace("\n", "") for l in fp.readlines()]

        if lines[0] in ("clientId,clientSecret", "client_id,client_secret"):
            client_id, client_secret = lines[1].split(",")
        else:
            print("First line read: '{}'".format(lines[0]))
            raise RuntimeError(
                "The specified file {} doesn't look like to be a Spark API client "
                "credentials file".format(file_path)
            )

    print(">>>> Found credentials!")
    print(
        ">>>> Client_id={}, client_secret={}****".format(client_id, client_secret[:5])
    )

    return client_id, client_secret


def do_api_post_query(uri, body, headers):
    """
    OAuth2 authentication requires a POST request with client credentials before accessing the API. 
    This POST request will return an Access Token which will be used for the API GET request.
    """
    url = urljoin(API_BASE_URL, uri)

    data = json.dumps(body).encode("utf-8")

    # HTTP POST request
    req = request.Request(url, data=data, headers=headers)
    try:
        response = request.urlopen(req)
    except HTTPError as e:
        print("HTTP Error: ", e.code)
        print(e.read())
        sys.exit(1)

    resp_content = response.read()

    # The server must return HTTP 201. Raise an error if this is not the case
    assert response.status == 201, resp_content

    # The server returned a JSON response
    content = json.loads(resp_content)

    return content


def do_api_get_query(uri, access_token):
    """
    After receiving an Access Token, we can request information from the API.
    """
    url = urljoin(API_BASE_URL, uri)

    headers = {
        "Authorization": "Bearer {}".format(access_token),
        "Accept": "application/json",
    }

    # HTTP POST request
    req = request.Request(url, headers=headers)
    try:
        response = request.urlopen(req)
    except HTTPError as e:
        print("HTTP Error: ", e.code)
        print(e.read())
        sys.exit(1)

    resp_content = response.read()

    # The server must return HTTP 201. Raise an error if this is not the case
    assert response.status == 200, resp_content

    # The server returned a JSON response
    content = json.loads(resp_content)

    return content


def get_access_token(client_id, client_secret):
    """
    Get a new access_token. Access tokens are the thing that applications use to make
    API requests. Access tokens must be kept confidential in storage.

    # Procedure:

    Do a POST query with `grantType` and `scopes` in the body. A basic authorization
    HTTP header is required. The "Basic" HTTP authentication scheme is defined in
    RFC 7617, which transmits credentials as `clientId:clientSecret` pairs, encoded
    using base64.
    """

    # Note: for the sake of this example, we choose to use the Python urllib from the
    # standard lib. One should consider using https://requests.readthedocs.io/

    payload = "{}:{}".format(client_id, client_secret).encode()
    headers = {
        "Authorization": b64encode(payload).decode(),
        "Accept": "application/json",
        "Content-Type": "application/json",
    }
    body = {
        "grantType": "clientCredentials",
        "scopes": "read:netbacks,read:access,read:prices,read:routes",
    }

    content = do_api_post_query(uri="/oauth/token/", body=body, headers=headers)

    print(
        ">>>> Successfully fetched an access token {}****, valid {} seconds.".format(
            content["accessToken"][:5], content["expiresIn"]
        )
    )

    return content["accessToken"]

## Defining Fetch Request

Here is where we define what type of data we want to fetch from the API.

In the fetch request, we use the URL:

__uri="/v1.0/netbacks/reference-data/"__

This query shows an overview on all available netbacks, showing all available ports and possible routes to/from these destinations (i.e. via Suez, Panama etc.).

An example of pulling all available Netbacks data for a specific route (e.g. Netbacks data for Sabine Pass via Suez) is shown later in this script. 

Pulling other data products (such as price releases) require different URL's in the fetch request (refer to other Python API examples).

In [37]:
# Define the function for listing all netbacks
def list_netbacks(access_token):
    """
    Fetch available routes. Return contract ticker symbols

    # Procedure:

    Do a GET query to /v1.0/routes/ with a Bearer token authorization HTTP header.
    """
    content = do_api_get_query(
        uri="/v1.0/netbacks/reference-data/", access_token=access_token
    )

    print(">>>> All the routes you can fetch")
    tickers = []
    fobPort_names = []

    availablevia = []

    for contract in content["data"]["staticData"]["fobPorts"]:
        tickers.append(contract["uuid"])
        fobPort_names.append(contract["name"])

        availablevia.append(contract["availableViaPoints"])

    reldates = content["data"]["staticData"]["sparkReleases"]

    dicto1 = content["data"]

    return tickers, fobPort_names, availablevia, reldates, dicto1

## N.B. Credentials

Here we call the above functions, and input the file path to our credentials.

N.B. You must have downloaded your client credentials CSV file before proceeding. Please refer to the API documentation if you have not dowloaded them already.  Instructions for downloading your credentials can be found here:

https://api.sparkcommodities.com/redoc#section/Authentication/Create-an-Oauth2-Client


In [38]:
# Input the path to your client credentials here
client_id, client_secret = retrieve_credentials(file_path="/tmp/client_credentials.csv")

# Authenticate:
access_token = get_access_token(client_id, client_secret)

# Fetch all contracts:
tickers, fobPort_names, availablevia, reldates, dicto1 = list_netbacks(access_token)

>>>> Found credentials!
>>>> Client_id=875f483b-19de-421a-8e9b-dceff6703e83, client_secret=6cdf8****
>>>> Successfully fetched an access token eyJhb****, valid 604799 seconds.
>>>> All the routes you can fetch


In [39]:
# Prints the callable route options, corresponding to each Route ID number shown above
# I.e. availablevia[2] shows the available route options for tickers[2]


print(availablevia)

[['suez', None], ['panama', None], ['cogh', 'panama', 'suez', None], ['suez', None], ['cogh', 'panama', 'suez', None], ['cogh', 'panama', 'suez', None], ['cogh', 'suez', None], ['cogh', 'suez', None], ['cogh', 'panama', None], ['cogh', 'panama', 'suez', None], ['cogh', 'panama', 'suez', None], ['panama', None], ['cogh', 'suez', None], ['suez', None], ['cogh', 'panama', 'suez', None], ['panama', None], ['cogh', 'suez', None], ['cogh', 'panama', 'suez', None], ['cogh', 'panama', 'suez', None], ['cogh', 'panama', 'suez', None], ['cogh', 'panama', 'suez', None], ['panama', None], ['cogh', 'suez', None], ['cogh', 'panama', 'suez', None], [None], ['cogh', None], ['cogh', 'panama', 'suez', None], ['cogh', 'panama', 'suez', None], ['panama', None], ['suez', None], [None], ['cogh', 'suez', None], ['cogh', 'panama', 'suez', None], ['cogh', 'suez', None]]


In [40]:
# Print the names of each of the ports, corresponding to Route ID and availablevia details shown above
# Some of these options are currently unavailable. 
# Please refer to the Netbacks tool on the Spark Platform to check which Netbacks are currently available

print(fobPort_names)

['Wheatstone', 'Woodfibre LNG', 'Corpus Christi', 'Bintulu', 'Lake Charles', 'Elba Island', 'Das Island', 'Gorgon', 'Atlantic LNG', 'Bethioua', 'Cove Point', 'Peru LNG', 'Ras Laffan', 'Murmansk', 'Hammerfest', 'LNG Canada', 'Yamal', 'Rio Grande LNG', 'Plaquemines', 'Altamira', 'Sabine Pass', 'Puerto Libertad', 'NWS', 'Delfin FLNG', 'Bioko', 'Bonny LNG', 'Freeport', 'Cameron (Liqu.)', 'Kamchatka', 'GLNG', 'Soyo', 'Qalhat', 'Calcasieu Pass', 'Tangguh']


In [41]:
# Shows the structure of the raw dictionary called
dicto1

{'staticData': {'viaPoints': [{'code': 'panama', 'name': 'Panama'},
   {'code': 'suez', 'name': 'Suez'},
   {'code': 'cogh', 'name': 'COGH'},
   {'code': 'magellan-straits', 'name': 'Strait of Magellan'}],
  'fobPorts': [{'uuid': '00398967-3ee1-4b26-bcdb-805ad19dbcce',
    'name': 'Wheatstone',
    'availableViaPoints': ['suez', None]},
   {'uuid': '00314d16-eada-4f37-bff3-d844708aeb45',
    'name': 'Woodfibre LNG',
    'availableViaPoints': ['panama', None]},
   {'uuid': '0030c461-9a63-403d-8f53-9327ea773517',
    'name': 'Corpus Christi',
    'availableViaPoints': ['cogh', 'panama', 'suez', None]},
   {'uuid': '003342b7-ba5b-4f0e-b6df-4d95837a5691',
    'name': 'Bintulu',
    'availableViaPoints': ['suez', None]},
   {'uuid': '003ff22f-77d8-413f-9997-2c6280e7c28c',
    'name': 'Lake Charles',
    'availableViaPoints': ['cogh', 'panama', 'suez', None]},
   {'uuid': '00352a22-e959-4233-b93d-d23a0da3dfed',
    'name': 'Elba Island',
    'availableViaPoints': ['cogh', 'panama', 'suez', N

### Reformatting

For a more accessible data format, we filter the data to only retrieve ports that have available Netbacks data. We then reformat this into a DataFrame.

In [42]:
# Define formatting data function
def format_store(available_via, fob_names, tickrs):
    dict_store = {
        "Index": [],
        "Callable Ports": [],
        "Corresponding Ticker": [],
        "Available Via": []
    }
    
    c = 0
    for a in available_via:
        ## Check which routes have non-empty Netbacks data and save indices
        if len(a) != 0:
            dict_store['Index'].append(c)

            # Use these indices to retrive the corresponding Netbacks info
            dict_store['Callable Ports'].append(fob_names[c])
            dict_store['Corresponding Ticker'].append(tickrs[c])
            dict_store['Available Via'].append(available_via[c])
        c += 1
    # Show available Netbacks ports in a DataFrame (with corresponding indices)
    dict_df = pd.DataFrame(dict_store)
    return dict_df

In [43]:
# Run formatting data function
available_df = format_store(availablevia,fobPort_names,tickers)

In [44]:
# View some of the dataframe
available_df.head()

,Index,Callable Ports,Corresponding Ticker,Available Via
0,0,Wheatstone,00398967-3ee1-4b26-bcdb-805ad19dbcce,"[suez, None]"
1,1,Woodfibre LNG,00314d16-eada-4f37-bff3-d844708aeb45,"[panama, None]"
2,2,Corpus Christi,0030c461-9a63-403d-8f53-9327ea773517,"[cogh, panama, suez, None]"
3,3,Bintulu,003342b7-ba5b-4f0e-b6df-4d95837a5691,"[suez, None]"
4,4,Lake Charles,003ff22f-77d8-413f-9997-2c6280e7c28c,"[cogh, panama, suez, None]"


## Fetching Netbacks Data specific to one port

Now that we can see all the available Netbacks data available to us, we can start to define what ports we want to call Netbacks data for (by referring to 'available_df' above).

The first step is to choose which port ID ('my_ticker') and which price release date ('my_release') we want. We check what possible routes are available for this port ('possible_via') and then choose one ('my_via').

__This is where you should input the specific Netbacks parameters you want to see__

In [45]:
# Choose route ID and price release date

# Here we define which port we want
port = "Cove Point"
ti = int(available_df[available_df["Callable Ports"] == port]["Index"])
my_ticker = tickers[ti]
my_release = reldates[0]

print(my_ticker)
print(my_release)

003e8539-3a98-48fa-b35d-0ba061beea4e
2024-10-17


/var/folders/v3/5pn2lssn077ch9xm2rttdmym0000gn/T/ipykernel_97073/215422504.py:5: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  ti = int(available_df[available_df["Callable Ports"] == port]["Index"])


In [46]:
# See possible route passage options
possible_via = availablevia[tickers.index(my_ticker)]
print(possible_via)

['cogh', 'panama', 'suez', None]


In [47]:
# Choose route passage
my_via = possible_via[0]
print(my_via)

cogh


## Defining the Function

We now define the function needed to call this data.

Again, we define what type of data we want to fetch from the API.

In the fetch request, we use the URL:

__uri="/v1.0/netbacks/"__

We then print the output.

__This function does not need to be altered by the user.__

In [48]:
## Defining the function


def fetch_netback(access_token, ticker, release, via=None, laden=None, ballast=None):
    """
    For a route, fetch then display the route details

    # Procedure:

    Do GET queries to https://api.sparkcommodities.com/v1.0/routes/{route_uuid}/
    with a Bearer token authorization HTTP header.
    """
    
    query_params = "?fob-port={}".format(ticker)
    if release is not None:
        query_params += "&release-date={}".format(release)
    if via is not None:
        query_params += "&via-point={}".format(via)
    if laden is not None:
        query_params += "&laden-congestion-days={}".format(laden)
    if ballast is not None:
        query_params += "&ballast-congestion-days={}".format(ballast)
    
    
    content = do_api_get_query(
        uri="/v1.0/netbacks/{}".format(query_params),
        access_token=access_token,
    )
    
    my_dict = content['data']

    return my_dict


## Calling that function and storing the output

my_dict = fetch_netback(access_token, my_ticker, release=my_release, via=my_via)

In [49]:
# Displaying the raw output
my_dict

{'name': 'Cove Point',
 'releaseDate': '2024-10-17',
 'viaPoint': 'cogh',
 'congestionLadenDays': 0,
 'congestionBallastDays': 0,
 'netbacks': [{'monthIdx': 1,
   'load': {'month': '2024-11', 'loadingDate': '2024-11-15'},
   'neaCargoDeliveryDate': '2024-12-27',
   'nweCargoDeliveryDate': '2024-11-29',
   'nea': {'outright': {'usdPerMMBtu': '11.729'},
    'ttfBasis': {'usdPerMMBtu': '-0.854'}},
   'nwe': {'outright': {'usdPerMMBtu': '11.867'},
    'ttfBasis': {'usdPerMMBtu': '-0.715'}},
   'max': {'outright': {'usdPerMMBtu': '11.867'},
    'ttfBasis': {'usdPerMMBtu': '-0.715'}},
   'neaMinusNwe': {'outright': {'usdPerMMBtu': '-0.138'},
    'ttfBasis': {'usdPerMMBtu': '-0.138'}},
   'neaMeta': {'ttfPrice': {'usdPerMMBtu': '12.777'},
    'ttfBasis': {'usdPerMMBtu': '0.757'},
    'desLngPrice': {'usdPerMMBtu': '13.534'},
    'routeCost': {'usdPerMMBtu': '0.944'},
    'volumeAdjustment': {'usdPerMMBtu': '-0.861'},
    'netbackOutright': {'usdPerMMBtu': '11.729'},
    'netbackTtfBasis': {'u

## Formatting

Here we format the raw output into a Pandas Dataframe. This is useful for plotting, netbacks analysis and comparisons.

In [50]:
# Defining function for storing and formating netback prices
def format_store_netback(dict_netback):
    dict_store = {
        "Period": [],
        "NEA Outrights": [],
        "NWE Outrights": [],
        "NEA TTF Basis": [],
        "NWE TTF Basis": [],
        "NEA-NWE Outrights": [],
        "NEA-NWE TTF Basis": []
    }

    route_name = dict_netback["name"]

    for m in dict_netback["netbacks"]:
        dict_store['Period'].append(m["load"]["month"])

        dict_store['NEA Outrights'].append(float(m["nea"]["outright"]["usdPerMMBtu"]))
        dict_store['NEA TTF Basis'].append(float(m["nea"]["ttfBasis"]["usdPerMMBtu"]))

        dict_store['NWE Outrights'].append(float(m["nwe"]["outright"]["usdPerMMBtu"]))
        dict_store['NWE TTF Basis'].append(float(m["nwe"]["ttfBasis"]["usdPerMMBtu"]))

        dict_store['NEA-NWE Outrights'].append(float(m["neaMinusNwe"]["outright"]["usdPerMMBtu"]))
        dict_store['NEA-NWE TTF Basis'].append(float(m["neaMinusNwe"]["ttfBasis"]["usdPerMMBtu"]))

    stored_df = pd.DataFrame(dict_store)
    return route_name, stored_df

In [51]:
# Store information in a DataFrame
route_name, netbacks_df = format_store_netback(my_dict)

netbacks_df.head()

,Period,NEA Outrights,NWE Outrights,NEA TTF Basis,NWE TTF Basis,NEA-NWE Outrights,NEA-NWE TTF Basis
0,2024-11,11.729,11.867,-0.854,-0.715,-0.138,-0.138
1,2024-12,11.748,11.920,-0.985,-0.813,-0.171,-0.171
2,2025-01,11.541,11.953,-1.305,-0.893,-0.413,-0.413
3,2025-02,11.130,11.888,-1.779,-1.021,-0.758,-0.758
4,2025-03,10.969,11.680,-1.828,-1.117,-0.711,-0.711


# Historical Data

Defining a function to fetch historical data and store as a Dataframe

The historical data call incorporates a "time.sleep(0.5)" function. This is needed if you're looking to pull more than 500 datapoints in an hour, as our API is rate-limited. For more information, please see our API Docs:

https://api.sparkcommodities.com/redoc#section/Requests-and-Responses/Rate-Limits

In [52]:
## Plotting Netbacks in a similar style to the Spark Platform Netbacks Price Chart

import time

def netbacks_history(tick, reldates, my_via=None, laden =None, ballast=None):

    months = []
    nea_outrights = []
    nea_ttfbasis = []
    nwe_outrights = []
    nwe_ttfbasis = []
    delta_outrights = []
    delta_ttfbasis = []
    release_date = []
    
    port = []

    for r in reldates:
        try:
            my_dict = fetch_netback(access_token, tickers[tick], release=r, via=my_via, laden=laden, ballast=ballast)
            
            for m in my_dict['netbacks']:

                months.append(m['load']['month'])

                nea_outrights.append(float(m['nea']['outright']['usdPerMMBtu']))
                nea_ttfbasis.append(float(m['nea']['ttfBasis']['usdPerMMBtu']))

                nwe_outrights.append(float(m['nwe']['outright']['usdPerMMBtu']))
                nwe_ttfbasis.append(float(m['nwe']['ttfBasis']['usdPerMMBtu']))

                delta_outrights.append(float(m['neaMinusNwe']['outright']['usdPerMMBtu']))
                delta_ttfbasis.append(float(m['neaMinusNwe']['ttfBasis']['usdPerMMBtu']))

                release_date.append(my_dict['releaseDate'])
                port.append(fobPort_names[tick])
                
        except:
            print('Bad Date: ' + r)
        
        # Incorporating sleep function
        time.sleep(0.5)
        
    historical_df = pd.DataFrame({
                'Release Date': release_date,
                'FoB Port': port,
                'Month': months,
                'NEA Outrights': nea_outrights,
                'NEA TTF Basis': nea_ttfbasis,
                'NWE Outrights': nwe_outrights,
                'NWE TTF Basis': nwe_ttfbasis,
                'Delta Outrights': delta_outrights,
                'Delta TTF Basis': delta_ttfbasis,
                })
    
    historical_df['Release Date'] = pd.to_datetime(historical_df['Release Date'])
    
    return historical_df


In [53]:
via ='cogh'
my_t = 'Sabine Pass'
t = fobPort_names.index(my_t)

my_rels = reldates[:3]

df_cogh = netbacks_history(t, my_rels, my_via='cogh')
df_panama = netbacks_history(t, my_rels, my_via='panama')

In [55]:
df_cogh.head()

,Release Date,FoB Port,Month,NEA Outrights,NEA TTF Basis,NWE Outrights,NWE TTF Basis,Delta Outrights,Delta TTF Basis
0,2024-10-17,Sabine Pass,2024-11,11.689,-0.893,11.763,-0.819,-0.074,-0.074
1,2024-10-17,Sabine Pass,2024-12,11.687,-1.046,11.788,-0.945,-0.100,-0.100
2,2024-10-17,Sabine Pass,2025-01,11.451,-1.395,11.807,-1.039,-0.356,-0.356
3,2024-10-17,Sabine Pass,2025-02,11.056,-1.853,11.718,-1.191,-0.662,-0.662
4,2024-10-17,Sabine Pass,2025-03,10.917,-1.880,11.494,-1.303,-0.577,-0.577
